<a href="https://colab.research.google.com/github/NahinAlam001/CSE-465/blob/model/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import os
import gdown
import zipfile
import torch
from torch import nn
from pathlib import Path
from tqdm.auto import tqdm
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo
from torchinfo import summary

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
gdown.download('https://drive.google.com/uc?id=1DLWHOlGDNeQNRRuonzaSgwNiT_J1NDx1', 'pcos.zip', quiet=False)
with zipfile.ZipFile('pcos.zip', 'r' ) as zip_ref:
  zip_ref.extractall()

data_path = Path('data')
image_path = data_path/'images'
train_dir = image_path/'train'
test_dir = image_path/'test'

Downloading...
From (original): https://drive.google.com/uc?id=1DLWHOlGDNeQNRRuonzaSgwNiT_J1NDx1
From (redirected): https://drive.google.com/uc?id=1DLWHOlGDNeQNRRuonzaSgwNiT_J1NDx1&confirm=t&uuid=a7e45624-374b-41f1-9150-427c1c0633dc
To: /content/pcos.zip
100%|██████████| 132M/132M [00:06<00:00, 21.5MB/s]


In [6]:
transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root = train_dir,
                                  transform = transform,
                                  target_transform = None)

test_data = datasets.ImageFolder(root = test_dir,
                                  transform = transform)

class_names = train_data.classes
class_names

['infected', 'notinfected']

In [10]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_loader = DataLoader(dataset = train_data,
                          batch_size = BATCH_SIZE,
                          shuffle = True,
                          num_workers = NUM_WORKERS)

test_loader = DataLoader(dataset = test_data,
                          batch_size = BATCH_SIZE,
                          shuffle = False,
                          num_workers = NUM_WORKERS)


batch_img, batch_label = next(iter(train_loader))
batch_img.shape, batch_label.shape

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


(torch.Size([32, 3, 64, 64]), torch.Size([32]))

In [24]:
class NahinGG(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_unit: int,
               output_shape: int)->None:

    super().__init__()

    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape,
                  out_channels = hidden_unit,
                  kernel_size = 3,
                  stride = 1,
                  padding = 0),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_unit,
                  out_channels = hidden_unit,
                  kernel_size = 3,
                  stride = 1,
                  padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,
                     stride = 2))

    self.conv_block2 = nn.Sequential(
        nn.Conv2d(in_channels = hidden_unit,
                  out_channels = hidden_unit,
                  kernel_size = 3,
                  stride = 1,
                  padding = 0),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_unit,
                  out_channels = hidden_unit,
                  kernel_size = 3,
                  stride = 1,
                  padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,
                     stride = 2))

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = 13*13*hidden_unit,
                  out_features = output_shape))

  def forward(self, x):
    return self.classifier(self.conv_block2(self.conv_block1(x)))

In [ ]:
model_0 = NahinGG(input_shape = 3,
                  hidden_unit = 10,
                  output_shape = len(class_names)).to(device)

summary(model_0, input_size = [1, 3, 64, 64])

In [30]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device = device
               ):

  model.train()

  train_loss, train_acc = 0, 0

  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #forward pass
    train_pred_logits = model(X)

    #calculate the loss
    loss = loss_fn(train_pred_logits, y)
    train_loss += loss.item()

    #optimizer zero grad
    optimizer.zero_grad()

    #backward propagation
    loss.backward()

    #optimizer step
    optimizer.step()

    train_pred_lables = torch.argmax(torch.softmax(train_pred_logits, dim=1), dim=1)
    train_acc += (train_pred_lables == y).sum().item()/len(train_pred_lables)

  #average loss and accuracy per batch
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc

In [36]:
def test_step(model = torch.nn.Module,
              dataloader = torch.utils.data.DataLoader,
              loss_fn = torch.nn.Module,
              device = device):

  model.eval()

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      test_pred_logits = model(X)

      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      test_pred_lables = torch.argmax(torch.softmax(test_pred_logits, dim=1), dim=1)
      test_acc += (test_pred_lables == y).sum().item()/len(test_pred_lables)

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

  return test_loss, test_acc

In [34]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int = 5,
          device = device):

  results = {
      "train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model = model,
                                       dataloader = train_dataloader,
                                       loss_fn = loss_fn,
                                       optimizer = optimizer,
                                       device = device)

    test_loss, test_acc = test_step(model = model,
                                    dataloader = test_dataloader,
                                    loss_fn = loss_fn,
                                    device = device)

    print(f'Epoch {epoch} | Train Loss: {train_loss: .4f} | Train Accuracy: {train_acc: .4f} | Test Loss: {test_loss: .4f} | Test Accuracy: {test_acc: .4f}')

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

In [38]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 1

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_0.parameters(),
                            lr = 1e-3)

from timeit import default_timer as timer

start_time = timer()
model_0_results = train(model = model_0,
                        train_dataloader = train_loader,
                        test_dataloader = test_loader,
                        loss_fn = loss_fn,
                        optimizer = optimizer,
                        epochs = NUM_EPOCHS,
                        device = device)
end_time = timer()

print(f'Total Training Time: {end_time - start_time: .3f} seconds')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0 | Train Loss:  0.0031 | Train Accuracy:  0.9990 | Test Loss:  0.0003 | Test Accuracy:  1.0000
Total Training Time:  16.830 seconds
